In [1]:
%pylab inline
from papers import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
pl, ts = arxiv_reading()
with open("./list_pkls/"+ts.split(",")[-1].strip().replace(" ", "_")+"_paper_list.pkl", "wb") as f:
    pck.dump(pl, f)
    
lp_c = ListPapers(pl)

New submissions for Thu, 10 Oct 19
Sleep...
Awake...


In [3]:
lp_c.summary()

Number of papers: 59
Subjects: 
['Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Astrophysics of Galaxies (astro-ph.GA)', 'Solar and Stellar Astrophysics (astro-ph.SR)', 'Fluid Dynamics (physics.flu-dyn)', 'Plasma Physics (physics.plasm-ph)', 'Earth and Planetary Astrophysics (astro-ph.EP)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'General Relativity and Quantum Cosmology (gr-qc)', 'Data Analysis, Statistics and Probability (physics.data-an)', 'Physics and Society (physics.soc-ph)', 'Systems and Control (eess.SY)', 'Robotics (cs.RO)', 'Space Physics (physics.space-ph)', 'High Energy Physics - Phenomenology (hep-ph)', 'Nuclear Theory (nucl-th)', 'Signal Processing (eess.SP)']
Key words: 

Exclude key words: 
Total scores:



In [4]:
lp_c.add_key_words(["Chemistry", "Galactic center", "SNR", "Supernova Remnant"])
lp_c.add_key_words("Cosmic Ray", boost=0.5)
lp_c.summary()


Number of papers: 59
Subjects: 
['Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Astrophysics of Galaxies (astro-ph.GA)', 'Solar and Stellar Astrophysics (astro-ph.SR)', 'Fluid Dynamics (physics.flu-dyn)', 'Plasma Physics (physics.plasm-ph)', 'Earth and Planetary Astrophysics (astro-ph.EP)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'General Relativity and Quantum Cosmology (gr-qc)', 'Data Analysis, Statistics and Probability (physics.data-an)', 'Physics and Society (physics.soc-ph)', 'Systems and Control (eess.SY)', 'Robotics (cs.RO)', 'Space Physics (physics.space-ph)', 'High Energy Physics - Phenomenology (hep-ph)', 'Nuclear Theory (nucl-th)', 'Signal Processing (eess.SP)']
Key words: 
['Chemistry', 'Galactic center', 'SNR', 'Supernova Remnant', 'Cosmic Ray']
Exclude key words: 
Total scores:
[0.0, 0.0, 0.0, 2.576116130761227, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.9305

In [5]:
lp_c.head()

ID: arXiv:1910.03631
Title: AutoSource: Radio-astronomical source-finding with convolutional  autoencoders
Authors: V. Lukic; F. De Gasperin; M. Brüggen
Date: New submissions for Thu, 10 Oct 19
Abstract: Finding and classifying astronomical sources is key in the scientific
exploitation of radio surveys. Source-finding usually involves identifying the
parts of an image belonging to an astronomical source, against some estimated
background. This can be problematic in the radio regime, owing to the presence
of correlated noise, which can interfere with the signal from the source. In
the current work we present AutoSource, a novel method based on a deep learning
technique, to identify the positions of radio sources, and compare the results
to a Gaussian-fitting method. Since the deep learning approach allows the
generation of more training images, it should perform well in the
source-finding task. We test the source-finding methods on artificial data
created for the data challenge of the S

In [6]:
lp_d = lp_c.filter_subjects('Earth and Planetary Astrophysics (astro-ph.EP)', exclude=True)
lp_d.summary()

Number of papers: 49
Subjects: 
['Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Astrophysics of Galaxies (astro-ph.GA)', 'Solar and Stellar Astrophysics (astro-ph.SR)', 'Fluid Dynamics (physics.flu-dyn)', 'Plasma Physics (physics.plasm-ph)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'General Relativity and Quantum Cosmology (gr-qc)', 'Data Analysis, Statistics and Probability (physics.data-an)', 'Physics and Society (physics.soc-ph)', 'Systems and Control (eess.SY)', 'Robotics (cs.RO)', 'Space Physics (physics.space-ph)', 'High Energy Physics - Phenomenology (hep-ph)', 'Nuclear Theory (nucl-th)', 'Signal Processing (eess.SP)']
Key words: 
['Chemistry', 'Galactic center', 'SNR', 'Supernova Remnant', 'Cosmic Ray']
Exclude key words: 
Total scores:
[0.0, 0.0, 0.0, 2.4131683757249043, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.6461472143419513, 0.0, 0.0, 0.0, 0.0, 1.9530323836709984, 0.0, 

In [7]:
lp_d.head(sync=True)

ID: arXiv:1910.03631
Title: AutoSource: Radio-astronomical source-finding with convolutional autoencoders
Authors: Lukic, V.; De Gasperin, F.; Brüggen, M.
Date: 2019/10/08
Abstract: Finding and classifying astronomical sources is key in the scientific
exploitation of radio surveys. Source-finding usually involves identifying the
parts of an image belonging to an astronomical source, against some estimated
background. This can be problematic in the radio regime, owing to the presence
of correlated noise, which can interfere with the signal from the source. In
the current work we present AutoSource, a novel method based on a deep learning
technique, to identify the positions of radio sources, and compare the results
to a Gaussian-fitting method. Since the deep learning approach allows the
generation of more training images, it should perform well in the
source-finding task. We test the source-finding methods on artificial data
created for the data challenge of the Square Kilometre Array 

In [8]:
print(lp_c._boost)

{'chemistry': 1.0, 'galactic center': 1.0, 'snr': 1.0, 'supernova remnant': 1.0, 'cosmic ray': 0.5}


In [9]:
lp_c[0].arxiv_id

'arXiv:1910.03580'